In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from tensorflow.keras.models import load_model

- argparse 모듈 사용

In [3]:
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", default=r'mask_data_preprocessed', help="input dataset")
ap.add_argument("-m", "--model", type=str, default="mask_detector_160x160.model", help="output model")
args = vars(ap.parse_args(args=[]))

- 학습률, 학습횟수, 배치사이즈를 설정

In [20]:
INIT_LR = 1e-4
epochs = 3
batch_size = 128

- imagepaths에 데이터셋의 모든 이미지를 가져옴

In [5]:
imagepaths = list(paths.list_images(args["dataset"]))

In [6]:
data = []
labels = []

- 이미지 경로에서 폴더 라벨로 자름
    - label = imagepath.split(os.path.sep)[-2]
- 이미지의 크기를 160으로 로드함.
    - image = load_img(imagepath, target_size = (160,160))
- 이미지를 array로 바꿈
    - image = img_to_array(image)
- 이미지 전처리 - *
    - image = preprocess_input(image)
- data에 image 를 저장하고 labels에 label 저장
    - data.append(image)
    - labels.append(label)

In [7]:
for imagepath in imagepaths:
    label = imagepath.split(os.path.sep)[-2]
    image = load_img(imagepath, target_size = (160,160))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

C:\Users\Administrator\anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


- data를 numpy 배열로 변경

- 참고 : https://notebook.community/junhwanjang/DataSchool/Lecture/13.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC/1)%20Scikit-Learn%EC%9D%98%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5

In [8]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

# Binarizer는 threshold 값을 기준으로 결과를 0,1로 구분한다. default는 0이다.
# LabelBinarizer는 원-핫 인코더와 유사하지만, 실제  사용된 클래스만 인코딩에 사용하며,
# 0/1 대신 사용할 값을 지정할 수 있다.
lb = LabelBinarizer()
# lb.fit_transform을 통해 labels에 저장된 폴더에 따라 라벨 값이 0 / 1 로 구분된다.
One_Hot_labels = lb.fit_transform(labels)
# to_categorical 을 통해 원 핫 벡터 형식으로 바꾼다. ( [0,1]  , [1,0])
One_Hot_labels = to_categorical(One_Hot_labels)

- train 세트와 test 세트에 8 : 2로 이미지 분할

In [9]:
(train_x,test_x,train_y,test_y) = train_test_split(data, One_Hot_labels, test_size=0.20, stratify = labels, random_state = 42)

- 데이터 증강 방법으로 해당 데이터의 전처리를 어떻게 할지 정함.
    - width_shift와 height_shift를 통해 좌우, 상하로 이미지를 늘림.
    - horizontal 을 통해 수평 반전.
    - rotation 을 통해 랜덤하게 각도 내의 범위만큼 돌림.
    - zoom 을 통해 랜덤하게 확대.
    - shear 를 통해 이미지를 변형시킴. 시계반대방향으로, 강도는 라디안.
    - fill_mode는 이미지 회전,이동 시 생기는 공간을 채우는 방식
- 참고 : https://srgai.tistory.com/16
- https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

In [10]:
datagen = ImageDataGenerator(rotation_range = 20,
                            zoom_range = 0.15,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.15,
                            horizontal_flip=True,
                            fill_mode = "nearest")

- 전이학습을 위한 FC LAYER를 제외한 MovileNetV2 생성
- 기존 오픈소스에서 연산 및 예측 부분에서 얼굴에 사용하기엔 224의 이미지가 크고 비효율적인 것 같아 160으로 이미지 크기를 줄이고, Pool_size 또한 5로 바꿈

In [11]:
baseModel = MobileNetV2(weights="imagenet",
                        include_top=False,
                        input_shape=(160,160,3), 
                        input_tensor=Input(shape=(160,160,3)))

- FC LAYER 추가.

- baseModel의 아웃풋을 TopModel로 받음
    - TopModel = baseModel.output
- 입력 이미지를 pool_size로 나누고 나눈 풀에서 평균값을 뽑아내 출력을 만듬.
    - TopModel = AveragePooling2D(pool_size=(5,5))(TopModel)
- 입력 이미지를 일차원으로 바꿔줌
    - TopModel = Flatten(name='flatten')(TopModel)
- flatten으로 일차원으로 바뀐 입력을 받고 출력뉴런을 128개로, 활성화 함수는 은닉층에서 주로 쓰는 relu를 사용
    - TopModel = Dense(128, activation='relu')(TopModel)
- dropout을 통해 뉴런의 절반을 생략, 줄어든 신경망을 통해 학습 수행할 수 있게함.
    - TopModel = Dropout(0.5)(TopModel)
- 마지막 LAYER는 분류결과를 계산하는 층으로 mask, no-mask를 위해 출력뉴련은 2개 이고, 활성화 함수는 카테고리 분류에 주로 쓰이는 softmax 사용.
    - TopModel = Dense(2, activation = 'softmax')(TopModel)
    

- 참고 : https://aidalab.tistory.com/22 , 
- https://m.blog.naver.com/PostView.nhn?blogId=laonple&logNo=220542170499&proxyReferer=https:%2F%2Fwww.google.com%2F
- https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/

In [12]:
TopModel = baseModel.output
TopModel = AveragePooling2D(pool_size=(5,5))(TopModel)
TopModel = Flatten(name='flatten')(TopModel)
TopModel = Dense(128, activation='relu')(TopModel)
TopModel = Dropout(0.5)(TopModel)
TopModel = Dense(2, activation = 'softmax')(TopModel)

- base 모델 위에 FC LAYER 추가

In [13]:
model = Model(inputs =baseModel.input, outputs = TopModel)

- base 모델은 학습하지 않음

In [14]:
for layer in baseModel.layers:
    layer.trainable = False

- loss 및 optimizer , metric 설정
- decay는 업데이트마다 적용되는 학습률의 감소율

In [15]:
model.compile(optimizer=Adam(lr=INIT_LR, decay=INIT_LR / epochs),
              loss='binary_crossentropy', 
              metrics=['accuracy'])

- 모델 학습시킴
- 증강방법을 통해 train셋을 배치사이즈로 에포크만큼 학습시킴.
- steps_per_epoch은 1epoch 당 몇번의 step을 진행할 것인지, 배치를 몇 번 학습시킬 것인지를 나타내며 주로 총 데이터셋/배치사이즈로 나타냄.
- validation_data 는 테스트 셋으로 진행.
- validation_step은 테스트셋/ 배치사이즈로 나타냈음.
- 모델 학습 과정 epochs=10, batch_size = 512 ==> epochs=20, batch_size=256 ==> epochs=3, batch_size = 128
- 모델 학습 결과 42/42 [==============================] - 54s 1s/step - loss: 0.0727 - accuracy: 0.9748 - val_loss: 0.0809 - val_accuracy: 0.9712

In [21]:
Hist = model.fit(datagen.flow(train_x, train_y, batch_size=batch_size),
                 steps_per_epoch = len(train_x) // batch_size,
                 validation_data = (test_x, test_y),
                 validation_steps= len(test_x) // batch_size,
                 epochs = epochs)

Epoch 1/3
42/42 [==============================] - 54s 1s/step - loss: 0.0788 - accuracy: 0.9733 - val_loss: 0.0810 - val_accuracy: 0.9734
Epoch 2/3
42/42 [==============================] - 54s 1s/step - loss: 0.0794 - accuracy: 0.9688 - val_loss: 0.0802 - val_accuracy: 0.9734
Epoch 3/3
42/42 [==============================] - 54s 1s/step - loss: 0.0727 - accuracy: 0.9748 - val_loss: 0.0809 - val_accuracy: 0.9712


- 테스트 예측해보기

In [22]:
predldxs = model.predict(test_x, batch_size = batch_size)
predldxs

array([[0.24081105, 0.75918895],
       [0.9974644 , 0.00253556],
       [0.01401506, 0.9859849 ],
       ...,
       [0.00176875, 0.99823123],
       [0.9971391 , 0.00286097],
       [0.00122431, 0.99877566]], dtype=float32)

- 확률이 제일 높은 예측 뽑기

In [93]:
predldxs = np.argmax(predldxs, axis =1)
predldxs

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

- 모델 저장

In [25]:
model.save(args['model'], save_format='h5')